<a href="https://colab.research.google.com/github/ThanaVi/THCovid19_Report/blob/master/ETL_THCovid19_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Target 
1. เปลี่ยนชื่อจังหวัดที่พบผู้ป่วย (Province of onset) เพื่อนำไปใช้ JOIN กับ ข้อมูล Latitude, Longtitude ที่เตรียมไว้

2. JOIN ข้อมูลเเล้วโหลดเข้า BigQuery

#Result

ข้อมูลจาก Source

1. ข้อมูลผู้ป่วยติดเชื้อ
![alt text](https://drive.google.com/uc?id=1p1ZMxni2BL__PzCzrcpPcZJ5QocxHmXw)
2. ข้อมูลละติจูด, ลองติจูดของแต่ละจังหวัดในประเทศไทยที่เตรียมไว้
![alt text](https://drive.google.com/uc?id=1t3goOwsWC7OAZd4ck8CSkP23I7rqKIZZ)

ข้อมูลที่ได้จาก Transformation
![alt text](https://drive.google.com/uc?id=1kIi-suj7SuI0R9yBTCfgHvFxQqT_pSPt)

#Install Pyspark

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xzvf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

!pip install pyspark

# Import library

In [0]:
from pyspark.sql import SparkSession, SQLContext, functions as fn

# Create spark session

In [0]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

# Extract



ไฟล์ข้อมูลผู้ป่วย Covid19

Source: https://data.go.th/dataset/covid-19-daily

In [0]:
case_df = spark.read.csv('/content/drive/My Drive/Project/Covid19/covid19_200112-200430.csv', header=True, inferSchema=True)
case_df.show()

+---+----+----+-----------+---------------------+-----------------+-------------+-----------------+-----------------+
| no| age| sex|nationality|Province of isolation|Notification date|Announce Date|Province of onset|District of onset|
+---+----+----+-----------+---------------------+-----------------+-------------+-----------------+-----------------+
|  1|61.0|หญิง|        จีน|                  กทม|             null|     12-01-20|              กทม|             null|
|  2|74.0|หญิง|        จีน|                  กทม|             null|     17-01-20|              กทม|             null|
|  3|73.0|หญิง|        ไทย|               นครปฐม|             null|     22-01-20|           นครปฐม|            เมือง|
|  4|68.0| ชาย|        จีน|                  กทม|             null|     22-01-20|              กทม|             null|
|  5|66.0|หญิง|        จีน|              นนทบุรี|             null|     24-01-20|              กทม|             null|
|  6|33.0|หญิง|        จีน|                  กทม|       

In [0]:
case_df.count()

3294

# Transform



##Problem Solving

ข้อมูลควรมี 2944 (จำนวนจาก Source) แต่อ่านจากไฟล์ได้ 3294 

In [0]:
case_df.collect()[2940:2950]

[Row(no=2941, age=62.0, sex='หญิง', nationality='ไทย', Province of isolation='ภูเก็ต', Notification date='28-04-20', Announce Date='29-04-20', Province of onset='ภูเก็ต', District of onset='ถลาง'),
 Row(no=2940, age=44.0, sex='หญิง', nationality='ไทย', Province of isolation='ภูเก็ต', Notification date='28-04-20', Announce Date='29-04-20', Province of onset='ภูเก็ต', District of onset='ถลาง'),
 Row(no=2945, age=30.0, sex='หญิง', nationality='ไทย', Province of isolation='สมุทรปราการ', Notification date='28-04-20', Announce Date='29-04-20', Province of onset='สมุทรปราการ', District of onset=None),
 Row(no=2946, age=22.0, sex='หญิง', nationality='ไทย', Province of isolation='สมุทรปราการ', Notification date='28-04-20', Announce Date='29-04-20', Province of onset='สมุทรปราการ', District of onset=None),
 Row(no=2939, age=8.0, sex='หญิง', nationality='ไทย', Province of isolation='กทม', Notification date='28-04-20', Announce Date='29-04-20', Province of onset='นนทบุรี', District of onset='เมือง

พบว่าที่เกินมาเป็น null

In [0]:
case_df.count() - case_df.where(case_df['no'].isNull()).count()

2947

สร้าง Dataframe ใหม่ โดยเลือกข้อมูลที่ไม่เป็น null

In [0]:
notNull_case_df = case_df.where(case_df['no'].isNotNull())
notNull_case_df.count()

2947

ได้ข้อมูลถูกต้อง

In [0]:
notNull_case_df.collect()[2940:2950]

[Row(no=2941, age=62.0, sex='หญิง', nationality='ไทย', Province of isolation='ภูเก็ต', Notification date='28-04-20', Announce Date='29-04-20', Province of onset='ภูเก็ต', District of onset='ถลาง'),
 Row(no=2940, age=44.0, sex='หญิง', nationality='ไทย', Province of isolation='ภูเก็ต', Notification date='28-04-20', Announce Date='29-04-20', Province of onset='ภูเก็ต', District of onset='ถลาง'),
 Row(no=2945, age=30.0, sex='หญิง', nationality='ไทย', Province of isolation='สมุทรปราการ', Notification date='28-04-20', Announce Date='29-04-20', Province of onset='สมุทรปราการ', District of onset=None),
 Row(no=2946, age=22.0, sex='หญิง', nationality='ไทย', Province of isolation='สมุทรปราการ', Notification date='28-04-20', Announce Date='29-04-20', Province of onset='สมุทรปราการ', District of onset=None),
 Row(no=2939, age=8.0, sex='หญิง', nationality='ไทย', Province of isolation='กทม', Notification date='28-04-20', Announce Date='29-04-20', Province of onset='นนทบุรี', District of onset='เมือง

## Check province name

เช็คชื่อจังหวัดที่มีว่าตรงกับชื่อจังหวัดของตารางที่จะ JOIN ด้วยหรือไม่

In [0]:
notNull_case_df.select('Province of onset').distinct().orderBy('Province of onset').count()

70

In [0]:
covid19_provinces = notNull_case_df.select('Province of onset').distinct().collect()
covid19_provinces[:10]

[Row(Province of onset='สุรินทร์'),
 Row(Province of onset='ระยอง'),
 Row(Province of onset='ลำปาง'),
 Row(Province of onset='กระบี่'),
 Row(Province of onset='ไม่ระบุ'),
 Row(Province of onset='บุรีรัมย์'),
 Row(Province of onset='สระแก้ว'),
 Row(Province of onset='ร้อยเอ็ด'),
 Row(Province of onset='ตาก'),
 Row(Province of onset='เพชรบูรณ์')]

List ของชื่อจังหวัดที่มีในข้อมูลผู้ป่วย Covid19

In [0]:
list_covid19_provinces = [covid19_provinces[i][0] for i in range(len(covid19_provinces))]
len(list_covid19_provinces)

70

โหลดข้อมูลสำหรับตรวจสอบชื่อจังหวัด

In [0]:
province_df = spark.read.csv('/content/drive/My Drive/Project/Covid19/single_province_lat_long.csv/part-00000-54915bbe-4d99-4979-bc9f-2d80487df96a-c000.csv', header=True, inferSchema=True)

In [0]:
province_df.count()

77

In [0]:
province_df.show()

+-----+--------------------+---------------+--------------------+
|CH_ID|          CHANGWAT_E|     CHANGWAT_T|            LAT_LONG|
+-----+--------------------+---------------+--------------------+
|   10|             Bangkok|  กรุงเทพมหานคร|13.75614285714286...|
|   11|       Samut Prakarn|    สมุทรปราการ|13.587,100.629384...|
|   12|          Nonthaburi|        นนทบุรี|13.85080000000000...|
|   13|        Pathum Thani|       ปทุมธานี|13.99642857142857...|
|   14|Phra Nakhon Si Ay...|พระนครศรีอยุธยา|14.35542857142857...|
|   15|           Ang Thong|        อ่างทอง|14.58299999999999...|
|   16|            Lop Buri|         ลพบุรี|14.78020833333333...|
|   17|           Sing Buri|      สิงห์บุรี|14.8995,100.40225...|
|   18|            Chai Nat|         ชัยนาท|15.18855555555555...|
|   19|            Saraburi|        สระบุรี|14.51227272727272...|
|   20|           Chon Buri|         ชลบุรี|13.34883333333333...|
|   21|              Rayong|          ระยอง|12.67957894736841...|
|   22|   

In [0]:
provinces = province_df.select('CHANGWAT_T').collect()

In [0]:
list_provinces = [provinces[i][0] for i in range(len(provinces))]

List ของชื่อจังหวัดที่จะนำมา JOIN ด้วย


In [0]:
list_provinces[:10]

['กรุงเทพมหานคร',
 'สมุทรปราการ',
 'นนทบุรี',
 'ปทุมธานี',
 'พระนครศรีอยุธยา',
 'อ่างทอง',
 'ลพบุรี',
 'สิงห์บุรี',
 'ชัยนาท',
 'สระบุรี']

In [0]:
for province in list_covid19_provinces:
  if province not in list_provinces:
    print(province)

ไม่ระบุ
None
กทม


มีชื่อจังหวัดที่ไม่ตรงกัน 3 ชื่อ 

เลือกแก้เฉพาะชื่อ กทม เป็น กรุงเทพมหานคร ตามตารางที่จะ JOIN ด้วย

In [0]:
notNull_case_df.withColumn('edited_Province of onset', fn.when(notNull_case_df['Province of onset'] == 'กทม', 'กรุงเทพมหานคร').otherwise(notNull_case_df['Province of onset'])).show()

+---+----+----+-----------+---------------------+-----------------+-------------+-----------------+-----------------+------------------------+
| no| age| sex|nationality|Province of isolation|Notification date|Announce Date|Province of onset|District of onset|edited_Province of onset|
+---+----+----+-----------+---------------------+-----------------+-------------+-----------------+-----------------+------------------------+
|  1|61.0|หญิง|        จีน|                  กทม|             null|     12-01-20|              กทม|             null|           กรุงเทพมหานคร|
|  2|74.0|หญิง|        จีน|                  กทม|             null|     17-01-20|              กทม|             null|           กรุงเทพมหานคร|
|  3|73.0|หญิง|        ไทย|               นครปฐม|             null|     22-01-20|           นครปฐม|            เมือง|                  นครปฐม|
|  4|68.0| ชาย|        จีน|                  กทม|             null|     22-01-20|              กทม|             null|           กรุงเทพมหานคร|

In [0]:
edited_case_df = notNull_case_df.withColumn('Province of onset', fn.when(notNull_case_df['Province of onset'] == 'กทม', 'กรุงเทพมหานคร').otherwise(notNull_case_df['Province of onset']))
edited_case_df.show()

+---+----+----+-----------+---------------------+-----------------+-------------+-----------------+-----------------+
| no| age| sex|nationality|Province of isolation|Notification date|Announce Date|Province of onset|District of onset|
+---+----+----+-----------+---------------------+-----------------+-------------+-----------------+-----------------+
|  1|61.0|หญิง|        จีน|                  กทม|             null|     12-01-20|    กรุงเทพมหานคร|             null|
|  2|74.0|หญิง|        จีน|                  กทม|             null|     17-01-20|    กรุงเทพมหานคร|             null|
|  3|73.0|หญิง|        ไทย|               นครปฐม|             null|     22-01-20|           นครปฐม|            เมือง|
|  4|68.0| ชาย|        จีน|                  กทม|             null|     22-01-20|    กรุงเทพมหานคร|             null|
|  5|66.0|หญิง|        จีน|              นนทบุรี|             null|     24-01-20|    กรุงเทพมหานคร|             null|
|  6|33.0|หญิง|        จีน|                  กทม|       

เลือกเฉพาะ column ที่ต้องการใช้

In [0]:
final_df = edited_case_df.select('no', 'age', 'sex', 'nationality',
                                 edited_case_df['Announce Date'].alias('date'), 
                                 edited_case_df['Province of onset'].alias('province'))

#Final


In [0]:
final_df.show()

+---+----+----+-----------+--------+---------------+
| no| age| sex|nationality|    date|       province|
+---+----+----+-----------+--------+---------------+
|  1|61.0|หญิง|        จีน|12-01-20|  กรุงเทพมหานคร|
|  2|74.0|หญิง|        จีน|17-01-20|  กรุงเทพมหานคร|
|  3|73.0|หญิง|        ไทย|22-01-20|         นครปฐม|
|  4|68.0| ชาย|        จีน|22-01-20|  กรุงเทพมหานคร|
|  5|66.0|หญิง|        จีน|24-01-20|  กรุงเทพมหานคร|
|  6|33.0|หญิง|        จีน|25-01-20|  กรุงเทพมหานคร|
|  7|57.0|หญิง|        จีน|26-01-20|  กรุงเทพมหานคร|
|  8|73.0|หญิง|        จีน|26-01-20|ประจวบคีรีขันธ์|
|  9|63.0| ชาย|        จีน|28-01-20|  กรุงเทพมหานคร|
| 10|28.0|หญิง|        จีน|28-01-20|        นนทบุรี|
| 11|33.0| ชาย|        จีน|28-01-20|  กรุงเทพมหานคร|
| 12|61.0| ชาย|        จีน|28-01-20|  กรุงเทพมหานคร|
| 13| 6.0| ชาย|        จีน|28-01-20|  กรุงเทพมหานคร|
| 14|32.0|หญิง|        จีน|28-01-20|         ภูเก็ต|
| 15|56.0| ชาย|        จีน|31-01-20|  กรุงเทพมหานคร|
| 16|50.0| ชาย|        ไทย|31-01-20|  กรุงเทพม

In [0]:
final_df.select('no').count()

2947

In [0]:
final_df.coalesce(1).write.csv('/content/drive/My Drive/Project/Covid19/single_covid19_data.csv', header=True)

#Load to BiqQuery

JOIN ข้อมูล Covid19 กับ Latitude, Longtitude ตามจังหวัด โดยใช้ Spark SQL

In [0]:
final_df.createOrReplaceTempView('covid19_data')

In [0]:
province_df.createOrReplaceTempView('th_provinces')

In [0]:
dataset = spark.sql('SELECT p.CHANGWAT_T province, count(c.no) case_count, p.LAT_LONG lat_long \
          FROM covid19_data c \
          RIGHT JOIN th_provinces p \
          ON c.province == p.CHANGWAT_T \
          GROUP BY p.CHANGWAT_T, p.LAT_LONG \
          ORDER BY case_count desc, province')
dataset.show()

+---------------+----------+--------------------+
|       province|case_count|            lat_long|
+---------------+----------+--------------------+
|  กรุงเทพมหานคร|      1477|13.75614285714286...|
|         ภูเก็ต|       216|7.800833333333332...|
|        นนทบุรี|       144|13.85080000000000...|
|    สมุทรปราการ|       118|13.587,100.629384...|
|           ยะลา|       116|6.547214285714285...|
|          สงขลา|        98|7.138857142857144...|
|        ปัตตานี|        93|6.854461538461539...|
|         ชลบุรี|        84|13.34883333333333...|
|      เชียงใหม่|        40|18.78853333333334...|
|       ปทุมธานี|        37|13.99642857142857...|
|       นราธิวาส|        33|6.387857142857142...|
|     ฉะเชิงเทรา|        21|13.71126315789473...|
|          ชุมพร|        21|10.44655882352941...|
|         นครปฐม|        21|13.82192,100.0324...|
|         กระบี่|        19|8.053903225806451...|
|           สตูล|        19|6.617406249999999...|
|     นครราชสีมา|        18|14.98143478260869...|


In [0]:
dataset.count()

77

In [0]:
dataset.coalesce(1).write.csv('./THCovid19_Data.csv', header=True)

เข้าใช้ GCP ด้วย CLI

In [0]:
! gcloud auth login

In [0]:
! gcloud config set project cobalt-bond-272904

Updated property [core/project].


In [0]:
! gsutil cp ./THCovid19_Data.csv/part-00000-5b79104b-e2d2-408a-a7a6-6335fab29deb-c000.csv gs://thanavi_project_data

Copying file://./THCovid19_Data.csv/part-00000-5b79104b-e2d2-408a-a7a6-6335fab29deb-c000.csv [Content-Type=text/csv]...
/ [1 files][  4.7 KiB/  4.7 KiB]                                                
Operation completed over 1 objects/4.7 KiB.                                      


In [0]:
! gsutil mv gs://thanavi_project_data/part-00000-5b79104b-e2d2-408a-a7a6-6335fab29deb-c000.csv gs://thanavi_project_data/THCovid19_Data.csv

Copying gs://thanavi_project_data/part-00000-5b79104b-e2d2-408a-a7a6-6335fab29deb-c000.csv [Content-Type=text/csv]...
Removing gs://thanavi_project_data/part-00000-5b79104b-e2d2-408a-a7a6-6335fab29deb-c000.csv...

Operation completed over 1 objects/4.7 KiB.                                      


In [0]:
! gsutil ls gs://thanavi_project_data

gs://thanavi_project_data/THCovid19_Data.csv


In [0]:
! bq ls

     datasetId     
 ----------------- 
  Starbuck_Stores  
  covid19          


In [0]:
! bq load --autodetect covid19.THCovid19_Data gs://thanavi_project_data/THCovid19_Data.csv

Waiting on bqjob_r274ea28a9dde1fab_00000171da10d348_1 ... (5s) Current status: DONE   


In [0]:
! bq head -n 10 covid19.THCovid19_Data

+---------------+------------+---------------------------------------+
|   province    | case_count |               lat_long                |
+---------------+------------+---------------------------------------+
| กรุงเทพมหานคร |       1477 | 13.75614285714286,100.475             |
| ภูเก็ต        |        216 | 7.800833333333332,98.36491666666666   |
| นนทบุรี       |        144 | 13.850800000000003,100.48519999999999 |
| สมุทรปราการ   |        118 | 13.587,100.62938461538464             |
| ยะลา          |        116 | 6.547214285714285,101.2362142857143   |
| สงขลา         |         98 | 7.138857142857144,100.59285714285714  |
| ปัตตานี       |         93 | 6.854461538461539,101.27084615384615  |
| ชลบุรี        |         84 | 13.348833333333333,100.98783333333336 |
| เชียงใหม่     |         40 | 18.78853333333334,98.99420000000002   |
| ปทุมธานี      |         37 | 13.996428571428572,100.5409285714286  |
+---------------+------------+---------------------------------------+
